In [ ]:
import os

from IPython.display import JSON
import s3fs

os.environ['AWS_PROFILE'] = 'waisman-admin'


In [ ]:
s3 = s3fs.S3FileSystem()
with s3.open('s3://nkalafut-celltrip/3gpu-2learn-3work.txt', 'r') as f:
    content = f.read()

In [ ]:
# TODO: Parse JSON string
JSON(content)